In [1]:
# Change values in cells with '+++++++ INPUT : ' as per need! Not valid for all cells, Paths are MUST!
# Python 3 has been used for 1_ and 2_ notebooks. For rest Python 2 has been used

In [2]:
# +++++++ INPUT : Train/Test files path +++++++
home_path='/home/pgoyal/Projects/kbc/'
train_name='train_df.csv'
test_name='test_df.csv'
mf_target_file = 'mf_target.csv'
cc_target_file = 'cc_target.csv'
cl_target_file = 'cl_target.csv'


## Data Loading

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload

In [4]:
# Supporting Scripts : Please keep in same folder
import basic_checks_funcs as hf                    # Refer : basic_checks_funcs.py
import feature_process_funcs as fpf                # Refer : feature_process_funcs.py

In [5]:
train_data = pd.read_csv(home_path+train_name, sep=',', date_parser=True)
test_data = pd.read_csv(home_path+test_name, sep=',', date_parser=True)

In [6]:
print("Train shape : " , train_data.shape)
print("Test shape : ", test_data.shape)

Train shape :  (969, 30)
Test shape :  (646, 30)


In [7]:
print("Train cols : ", train_data.columns)
print(" ---- ")
print("Test cols : " ,test_data.columns)

Train cols :  Index(['Client', 'Sex', 'Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF',
       'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA',
       'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred',
       'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb',
       'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder'],
      dtype='object')
 ---- 
Test cols :  Index(['Client', 'Sex', 'Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF',
       'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA',
       'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred',
       'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb',
       'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_P

In [8]:
# Check for all columns not in Test, but in Train
list(set(train_data) - set(test_data))

[]

In [9]:
train_data.iloc[1,:]

Client                              850
Sex                                   F
Age                                  49
Tenure                               44
Count_CA                              1
Count_SA                            NaN
Count_MF                            NaN
Count_OVD                           NaN
Count_CC                            NaN
Count_CL                            NaN
ActBal_CA                       43.5232
ActBal_SA                           NaN
ActBal_MF                           NaN
ActBal_OVD                          NaN
ActBal_CC                           NaN
ActBal_CL                           NaN
VolumeCred                       1392.4
VolumeCred_CA                    1392.4
TransactionsCred                      3
TransactionsCred_CA                   3
VolumeDeb                       1226.35
VolumeDeb_CA                    1226.35
VolumeDebCash_Card                    0
VolumeDebCashless_Card                0
VolumeDeb_PaymentOrder          121.929


In [10]:
# Read target Files
mf_target = pd.read_csv(home_path+mf_target_file, sep=',', date_parser=True)
cc_target = pd.read_csv(home_path+cc_target_file, sep=',', date_parser=True)
cl_target = pd.read_csv(home_path+cl_target_file, sep=',', date_parser=True)

## Basic Checks and Column Shortlisting (for imputation)

In [11]:
# +++++++ INPUT : Dropping first unrequired columns <Syntax> : Train(ID, Target), Test(ID) as of now +++++++
train_drop = ['Client']
test_drop = ['Client']

In [12]:
# Shortlisting the ID cols
train_id = pd.DataFrame(train_data[train_drop[0]])
train_id.rename(columns={ train_drop[0]: 'ID_Col'}, inplace=True)
test_id = pd.DataFrame(test_data[test_drop[0]])
test_id.rename(columns={ test_drop[0]: 'ID_Col'}, inplace=True)

In [13]:
print("Total Rows in Train :", train_data.shape[0], " and Total unique IDs in Train : ", train_id['ID_Col'].nunique() )
print("Total Rows in Test :", test_data.shape[0], " and Total unique IDs in Test : ", test_id['ID_Col'].nunique() )

Total Rows in Train : 969  and Total unique IDs in Train :  969
Total Rows in Test : 646  and Total unique IDs in Test :  646


In [14]:
# +++++ Handle rare case of repetition of ID cols ++++++++

# Depends on the problem type! : Keeping the first occurrence here:
train_data.drop_duplicates(train_drop[0], keep='first', inplace=True)
test_data.drop_duplicates(test_drop[0], keep='first', inplace=True)

In [15]:
print("Train shape : " , train_data.shape)
print("Test shape : " , test_data.shape)

Train shape :  (969, 30)
Test shape :  (646, 30)


In [16]:
# ['Sale_XX','Revenue_XX']
mf_target_cols = ['Client','Sale_MF','Revenue_MF']   # Target columns for Mutual Fund
cc_target_cols = ['Client','Sale_CC','Revenue_CC']   # Target columns for Credit Card
cl_target_cols = ['Client','Sale_CL','Revenue_CL']   # Target columns for Consumer Loan

In [17]:
# Target Info
mf_target.rename(columns={ mf_target_cols[1]: 'Target'}, inplace=True)
print("Total Rows in MF Target :", mf_target.shape[0])
print("Unique count of each flag : \n", mf_target['Target'].value_counts(dropna=False) )

Total Rows in MF Target : 969
Unique count of each flag : 
 0    776
1    193
Name: Target, dtype: int64


In [18]:
# Target Info
cc_target.rename(columns={ cc_target_cols[1]: 'Target'}, inplace=True)
print("Total Rows in CC Target :",cc_target.shape[0])
print("Unique count of each flag : \n", cc_target['Target'].value_counts(dropna=False) )

Total Rows in CC Target : 969
Unique count of each flag : 
 0    727
1    242
Name: Target, dtype: int64


In [19]:
# Target Info
cl_target.rename(columns={ cl_target_cols[1]: 'Target'}, inplace=True)
print("Total Rows in CL Target :", cl_target.shape[0])
print("Unique count of each flag : \n", cl_target['Target'].value_counts(dropna=False) )

Total Rows in CL Target : 969
Unique count of each flag : 
 0    679
1    290
Name: Target, dtype: int64


In [20]:
# Redefining the ID and Target columns in Train and Test and saving in the below dataframes : train_id and test_id
train_id = pd.DataFrame(train_data[train_drop[0]])
train_id.rename(columns={ train_drop[0]: 'ID_Col'}, inplace=True)
train_id.reset_index(drop=True, inplace=True)

test_id = pd.DataFrame(test_data[test_drop[0]])
test_id.rename(columns={ test_drop[0]: 'ID_Col'}, inplace=True)
test_id.reset_index(drop=True, inplace=True)

In [21]:
# Removing the columns with only 1 unique value (Doing this after above operations, else will include nan/null too)
for cols in train_data.columns:
    if(len(train_data[cols].unique())==1):
        train_data.drop(cols,axis=1,inplace=True)
print(train_data.columns)

Index(['Client', 'Sex', 'Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF',
       'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA',
       'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred',
       'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb',
       'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder'],
      dtype='object')


In [22]:
print('MF target',mf_target.shape)
print('CC target',cc_target.shape)
print('CL target',cl_target.shape)
print('----')
print('Train ID',train_id.shape)
print('Test ID',test_id.shape)

MF target (969, 3)
CC target (969, 3)
CL target (969, 3)
----
Train ID (969, 1)
Test ID (646, 1)


In [23]:
train_data.drop(train_drop, inplace=True, axis=1)
test_data.drop(test_drop, inplace=True, axis=1)
del train_drop,test_drop

In [24]:
# Assuring test to have all train cols :
test_data = test_data[list(train_data.columns)]
print("Train shape : " , train_data.shape)
print("Test shape : ", test_data.shape)

Train shape :  (969, 29)
Test shape :  (646, 29)


In [25]:
# Shortlist the cols_ lists if only selected columns have to be evaluated
train_data.dtypes

Sex                              object
Age                               int64
Tenure                            int64
Count_CA                          int64
Count_SA                        float64
Count_MF                        float64
Count_OVD                       float64
Count_CC                        float64
Count_CL                        float64
ActBal_CA                       float64
ActBal_SA                       float64
ActBal_MF                       float64
ActBal_OVD                      float64
ActBal_CC                       float64
ActBal_CL                       float64
VolumeCred                      float64
VolumeCred_CA                   float64
TransactionsCred                float64
TransactionsCred_CA             float64
VolumeDeb                       float64
VolumeDeb_CA                    float64
VolumeDebCash_Card              float64
VolumeDebCashless_Card          float64
VolumeDeb_PaymentOrder          float64
TransactionsDeb                 float64


In [26]:
# Creating feature list on the basis of type :
uni_dtype = np.unique(train_data.dtypes)
for i in range(0,len(uni_dtype)):
    vars()['cols_'+str(uni_dtype[i])] = list(train_data.select_dtypes(include=[uni_dtype[i]]).columns)

In [27]:
print("Initial set of different type of columns :")
for i in range(0,len(uni_dtype)):
    print(vars()['cols_'+str(uni_dtype[i])])

Initial set of different type of columns :
['Age', 'Tenure', 'Count_CA']
['Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder']
['Sex']


In [28]:
# Check!
hf.get_missing_info('MF Target Data',mf_target,['Target'] )                 # Refer : basic_checks_funcs.py
print("---")
hf.get_missing_info('CC Target Data',cc_target,['Target'] )                 # Refer : basic_checks_funcs.py
print("---")
hf.get_missing_info('CL Target Data',cl_target,['Target'] )                 # Refer : basic_checks_funcs.py
print("---")

Total records in  MF Target Data 969

Total number of columns with missing value in MF Target Data :0
---
Total records in  CC Target Data 969

Total number of columns with missing value in CC Target Data :0
---
Total records in  CL Target Data 969

Total number of columns with missing value in CL Target Data :0
---


In [29]:
cols_var=[]
for i in range(0,len(uni_dtype)):
    cols_var.append('cols_%s'%str(uni_dtype[i]))
print(cols_var)

['cols_int64', 'cols_float64', 'cols_object']


In [30]:
# Replacing the blank values in these columns by 0, as these are mainly count and account balance values, so imputing by 0
count_cols = ['Count_CA','Count_SA','Count_MF','Count_OVD','Count_CC','Count_CL','ActBal_CA','ActBal_SA','ActBal_MF','ActBal_OVD','ActBal_CC','ActBal_CL']
for impute_cols in count_cols :
    train_data[impute_cols].fillna(0, inplace=True)

In [31]:
train_data.columns

Index(['Sex', 'Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD',
       'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF',
       'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA',
       'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder'],
      dtype='object')

In [32]:
# Refer : basic_checks_funcs.py

for i in range(0,len(cols_var)):
    print('\n -----------', str(uni_dtype[i]),'Variables-----------')
    vars()['new_'+str(uni_dtype[i])], vars()['imp_train_'+str(uni_dtype[i])]=  hf.get_missing_info('Training Data', train_data, vars()['cols_'+str(uni_dtype[i])] )
    
    print("\n Shortlisted", str(uni_dtype[i]),"variables :", vars()['new_'+str(uni_dtype[i])], "\n")
    
    # Passing only these shortlisted variables for the test dataset
    _, vars()['imp_test_'+str(uni_dtype[i])]  = hf.get_missing_info('Test Data', test_data, vars()['new_'+str(uni_dtype[i])] )
    


 ----------- int64 Variables-----------
Total records in  Training Data 969

Total number of columns with missing value in Training Data :0

 Shortlisted int64 variables : ['Age', 'Tenure', 'Count_CA'] 

Total records in  Test Data 646

Total number of columns with missing value in Test Data :0

 ----------- float64 Variables-----------
Total records in  Training Data 969

-- Column : VolumeCred --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeCred_CA --
Missing Count : 18
Missing Percent: 1.858

-- Column : TransactionsCred --
Missing Count : 18
Missing Percent: 1.858

-- Column : TransactionsCred_CA --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeDeb --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeDeb_CA --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeDebCash_Card --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeDebCashless_Card --
Missing Count : 18
Missing Percent: 1.858

-- Column : VolumeDeb_PaymentOrd

In [33]:
final_vars = []
for i in range(0,len(uni_dtype)):
    print("Final" ,str(uni_dtype[i]), "Type variables to be taken :\n ", vars()['new_'+str(uni_dtype[i])], "\n ")
    final_vars.append(vars()['new_'+str(uni_dtype[i])])

final_vars = [i for sub in final_vars for i in sub]
print("Overall Final variables to be taken : \n",final_vars)

Final int64 Type variables to be taken :
  ['Age', 'Tenure', 'Count_CA'] 
 
Final float64 Type variables to be taken :
  ['Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder'] 
 
Final object Type variables to be taken :
  ['Sex'] 
 
Overall Final variables to be taken : 
 ['Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDe

In [34]:
print("Columns to be imputed in Train ")
for i in range(0,len(uni_dtype)):
    print( str(uni_dtype[i]), "Type :", vars()['imp_train_'+str(uni_dtype[i])] )

Columns to be imputed in Train 
int64 Type : []
float64 Type : ['VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder']
object Type : ['Sex']


In [35]:
print("Columns to be imputed in Test ")
for i in range(0,len(uni_dtype)):
    print( str(uni_dtype[i]), "Type :",  vars()['imp_test_'+str(uni_dtype[i])] )

Columns to be imputed in Test 
int64 Type : []
float64 Type : ['Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder']
object Type : ['Sex']


In [36]:
train_data = train_data[final_vars]
test_data = test_data[final_vars]

train_data.iloc[0][final_vars]

Age                                     38
Tenure                                 165
Count_CA                                 1
Count_SA                                 0
Count_MF                                 0
Count_OVD                                0
Count_CC                                 0
Count_CL                                 0
ActBal_CA                          6752.24
ActBal_SA                                0
ActBal_MF                                0
ActBal_OVD                               0
ActBal_CC                                0
ActBal_CL                                0
VolumeCred                      0.00214286
VolumeCred_CA                   0.00214286
TransactionsCred                         1
TransactionsCred_CA                      1
VolumeDeb                          714.286
VolumeDeb_CA                       714.286
VolumeDebCash_Card                       0
VolumeDebCashless_Card                   0
VolumeDeb_PaymentOrder             714.286
Transaction

In [37]:
uni_dtype

array([dtype('int64'), dtype('float64'), dtype('O')], dtype=object)

In [38]:
# +++++++ INPUT : Column Types ++++++++++ Generalize this part for Date Columns!
cont_vars = vars()['new_'+str(uni_dtype[0])]                                             # Final Continuous Variables
flt_vars = vars()['new_'+str(uni_dtype[1])]                                              # Final Float Variables
cate_vars = vars()['new_'+str(uni_dtype[2])]                                             # Final Categorical Variables

In [39]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [40]:
cont_vars

['Age', 'Tenure', 'Count_CA']

## Feature Creation and Processing

### Categorical Variables : 

In [41]:
#1. shortlist dataframe with these variables only
#2. print count of unique values
#3. bucket all values in 'others' if less than 5% or missing
#4. print distribution again
#5. split in dummies and combine to the data frame again

In [42]:
vars()['imp_train_'+str(uni_dtype[2])]

['Sex']

In [43]:
# List of categorical features to be imputed in Train and Test  :
#imp_train_object = [i for i in vars()['imp_train_'+str(uni_dtype[1])] if i not in date_vars]
#imp_test_object = [i for i in vars()['imp_test_'+str(uni_dtype[1])] if i not in date_vars]

In [44]:
# Impute the missing value
def cat_imp(data, cols):
    for col in cols:
        data[col]= data[col].fillna(data[col].mode().item())
    return data
train_data = cat_imp(train_data, vars()['imp_train_'+str(uni_dtype[2])] )
test_data = cat_imp(test_data, vars()['imp_test_'+str(uni_dtype[2])] )

In [45]:
# Select only those categorical which have unique entries <=10, and use them for dummies, and rest for encode
def cat_len(train,test, cats):
    df = []
    for col in cats:
        df.append({'Column': col, 'Unique Train Count': train[col].nunique(), 'Unique Test Count': test[col].nunique() })
    return pd.DataFrame(df)

cat_df = cat_len(train_data, test_data, cate_vars)
cat_df

Column  Unique Test Count  Unique Train Count
0    Sex                  2                   2

In [46]:
# Features to be used for dummy : but only 1 column to be made
cat_dum_2 = list(cat_df['Column'][(cat_df['Unique Train Count']==2) & (cat_df['Unique Test Count']==2) ])

#### 1. Get_Dummies Set

In [47]:
# Create dummies  : Above variables ( cat_dum, cat_dum_2) on basis of the unique count in their columns

# Pass categorical vars with unique value=2                                  # Refer : feature_process_funcs.py
train_data = fpf.dummy_fn(train_data, cat_dum_2, 2)
test_data = fpf.dummy_fn(test_data, cat_dum_2, 2)

In [48]:
print("Final Columns after 1st operation \n Train :\n Columns :", train_data.columns,"\n Shape :",train_data.shape,
      "\n\n Test :\n Columns :", test_data.columns,"\n Shape",test_data.shape)
#del cat_dum
#del cat_dum_2

Final Columns after 1st operation 
 Train :
 Columns : Index(['Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD',
       'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF',
       'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA',
       'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder', 'Sex_F'],
      dtype='object') 
 Shape : (969, 29) 

 Test :
 Columns : Index(['Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD',
       'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF',
       'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA',
       'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeD

In [49]:
train_data.head()

Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  Count_CC  Count_CL  \
0   38     165         1       0.0       0.0        0.0       0.0       0.0   
1   49      44         1       0.0       0.0        0.0       0.0       0.0   
2   54      34         1       1.0       0.0        0.0       1.0       1.0   
3   29     106         1       0.0       0.0        0.0       0.0       0.0   
4   14     187         1       0.0       0.0        1.0       0.0       0.0   

     ActBal_CA     ActBal_SA  ...    VolumeDeb_CA  VolumeDebCash_Card  \
0  6752.244643      0.000000  ...      714.285714            0.000000   
1    43.523214      0.000000  ...     1226.345357            0.000000   
2    29.024286  14447.801429  ...     3794.580714          357.142857   
3    27.035714      0.000000  ...        0.000000            0.000000   
4   345.686071      0.000000  ...      130.521429            0.000000   

   VolumeDebCashless_Card  VolumeDeb_PaymentOrder  TransactionsDeb  \
0                0.000000              714.285714              1.0   
1                0.000000              121.928571              6.0   
2              444.597500             2076.785714             48.0   
3                0.000000                0.000000              0.0   
4              111.235714               19.285714              5.0   

   TransactionsDeb_CA  TransactionsDebCash_Card  TransactionsDebCashless_Card  \
0                 1.0                       0.0                           0.0   
1                 6.0                       0.0                           0.0   
2                38.0                       1.0                          26.0   
3                 0.0                       0.0                           0.0   
4                 5.0                       0.0                           4.0   

   TransactionsDeb_PaymentOrder  Sex_F  
0                           1.0      0  
1                           1.0      1  
2                          11.0      0  
3                           0.0      0  
4                           1.0      0  

[5 rows x 29 columns]

In [50]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

### Continuous Variables

In [51]:
#1. shortlist dataframe with these variables only
#2. Impute missing by mean
#3. plot distribution again for all variables
#4. check for any outliers : handle those
#5. if ok then combine to the data frame again

In [52]:
# Initial missing value counts:
print("Present values in Train Dataset of shape-",train_data.shape," before Imputation :\n", train_data[cont_vars].count())
print("Present values in Test Dataset of shape-",test_data.shape," before Imputation :\n", test_data[cont_vars].count())

Present values in Train Dataset of shape- (969, 29)  before Imputation :
 Age         969
Tenure      969
Count_CA    969
dtype: int64
Present values in Test Dataset of shape- (646, 29)  before Imputation :
 Age         646
Tenure      646
Count_CA    646
dtype: int64


In [53]:
# Imputation for Continuous Variables   
train_data = fpf.cnt_shortlist(train_data,flt_vars,"Train Dataset")     # Refer : feature_process_funcs.py
test_data = fpf.cnt_shortlist(test_data,flt_vars,"Test Dataset")        # Refer : feature_process_funcs.py

Train Dataset has variables ['VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder'] with missing values
Test Dataset has variables ['Count_SA', 'Count_MF', 'Count_OVD', 'Count_CC', 'Count_CL', 'ActBal_SA', 'ActBal_MF', 'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA', 'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA', 'VolumeDebCash_Card', 'VolumeDebCashless_Card', 'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA', 'TransactionsDebCash_Card', 'TransactionsDebCashless_Card', 'TransactionsDeb_PaymentOrder'] with missing values


In [54]:
print(train_data.shape)
print(test_data.shape)

(969, 29)
(646, 29)


In [55]:
# Final missing value counts:
print("Present values in Train Dataset of shape-",train_data.shape," after Imputation :\n", train_data[cont_vars].count())
print("Present values in Test Dataset of shape-",test_data.shape," after Imputation :\n", test_data[cont_vars].count())

Present values in Train Dataset of shape- (969, 29)  after Imputation :
 Age         969
Tenure      969
Count_CA    969
dtype: int64
Present values in Test Dataset of shape- (646, 29)  after Imputation :
 Age         646
Tenure      646
Count_CA    646
dtype: int64


In [56]:
print("Final Columns after 2nd operation Train : Columns :", train_data.columns," Shape :",train_data.shape,
      " Test :\n Columns :", test_data.columns,"\n Shape",test_data.shape)

Final Columns after 2nd operation Train : Columns : Index(['Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD',
       'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF',
       'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA',
       'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeDebCashless_Card',
       'VolumeDeb_PaymentOrder', 'TransactionsDeb', 'TransactionsDeb_CA',
       'TransactionsDebCash_Card', 'TransactionsDebCashless_Card',
       'TransactionsDeb_PaymentOrder', 'Sex_F'],
      dtype='object')  Shape : (969, 29)  Test :
 Columns : Index(['Age', 'Tenure', 'Count_CA', 'Count_SA', 'Count_MF', 'Count_OVD',
       'Count_CC', 'Count_CL', 'ActBal_CA', 'ActBal_SA', 'ActBal_MF',
       'ActBal_OVD', 'ActBal_CC', 'ActBal_CL', 'VolumeCred', 'VolumeCred_CA',
       'TransactionsCred', 'TransactionsCred_CA', 'VolumeDeb', 'VolumeDeb_CA',
       'VolumeDebCash_Card', 'VolumeDebCash

In [57]:
train_data.head()

Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  Count_CC  Count_CL  \
0   38     165         1       0.0       0.0        0.0       0.0       0.0   
1   49      44         1       0.0       0.0        0.0       0.0       0.0   
2   54      34         1       1.0       0.0        0.0       1.0       1.0   
3   29     106         1       0.0       0.0        0.0       0.0       0.0   
4   14     187         1       0.0       0.0        1.0       0.0       0.0   

     ActBal_CA     ActBal_SA  ...    VolumeDeb_CA  VolumeDebCash_Card  \
0  6752.244643      0.000000  ...      714.285714            0.000000   
1    43.523214      0.000000  ...     1226.345357            0.000000   
2    29.024286  14447.801429  ...     3794.580714          357.142857   
3    27.035714      0.000000  ...        0.000000            0.000000   
4   345.686071      0.000000  ...      130.521429            0.000000   

   VolumeDebCashless_Card  VolumeDeb_PaymentOrder  TransactionsDeb  \
0                0.000000              714.285714              1.0   
1                0.000000              121.928571              6.0   
2              444.597500             2076.785714             48.0   
3                0.000000                0.000000              0.0   
4              111.235714               19.285714              5.0   

   TransactionsDeb_CA  TransactionsDebCash_Card  TransactionsDebCashless_Card  \
0                 1.0                       0.0                           0.0   
1                 6.0                       0.0                           0.0   
2                38.0                       1.0                          26.0   
3                 0.0                       0.0                           0.0   
4                 5.0                       0.0                           4.0   

   TransactionsDeb_PaymentOrder  Sex_F  
0                           1.0      0  
1                           1.0      1  
2                          11.0      0  
3                           0.0      0  
4                           1.0      0  

[5 rows x 29 columns]

In [58]:
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Final Cleaned Datasets

In [59]:
# Final missing value counts:
print("Present values in Train Dataset of shape-",train_data.shape," before Modeling :\n", train_data.count())
print("Present values in Test Dataset of shape-",test_data.shape," before Modeling :\n", test_data.count())

Present values in Train Dataset of shape- (969, 29)  before Modeling :
 Age                             969
Tenure                          969
Count_CA                        969
Count_SA                        969
Count_MF                        969
Count_OVD                       969
Count_CC                        969
Count_CL                        969
ActBal_CA                       969
ActBal_SA                       969
ActBal_MF                       969
ActBal_OVD                      969
ActBal_CC                       969
ActBal_CL                       969
VolumeCred                      969
VolumeCred_CA                   969
TransactionsCred                969
TransactionsCred_CA             969
VolumeDeb                       969
VolumeDeb_CA                    969
VolumeDebCash_Card              969
VolumeDebCashless_Card          969
VolumeDeb_PaymentOrder          969
TransactionsDeb                 969
TransactionsDeb_CA              969
TransactionsDebCash_Card    

In [60]:
print('MF target',mf_target.shape)
print('CC target',cc_target.shape)
print('CL target',cl_target.shape)
print('----')
print('Train ID',train_id.shape)
print('Test ID',test_id.shape)
print('----')
print('Train Data',train_data.shape)
print('Test Data',test_data.shape)

MF target (969, 3)
CC target (969, 3)
CL target (969, 3)
----
Train ID (969, 1)
Test ID (646, 1)
----
Train Data (969, 29)
Test Data (646, 29)


In [61]:
# Mutual Fund Final Dataframe
train_mf_final = [train_id, train_data, mf_target[[s for s in mf_target.columns if "Client" not in s]]]
train_mf_final = pd.concat(train_mf_final, axis=1)
print(train_mf_final.shape)

# Credit Card Final Dataframe
train_cc_final = [train_id, train_data, cc_target[[s for s in cc_target.columns if "Client" not in s]]]
train_cc_final = pd.concat(train_cc_final, axis=1)
print(train_cc_final.shape)

# Consumer Loan Final Dataframe
train_cl_final = [train_id, train_data, cl_target[[s for s in cl_target.columns if "Client" not in s]]]
train_cl_final = pd.concat(train_cl_final, axis=1)
print(train_cl_final.shape)

(969, 32)
(969, 32)
(969, 32)


In [62]:
test_final = [test_id, test_data]
test_final = pd.concat(test_final, axis=1)
print(test_final.shape)
test_final.head()

(646, 30)


ID_Col  Age  Tenure  Count_CA  Count_SA  Count_MF  Count_OVD  Count_CC  \
0     909   21      27         1  1.142857  3.918033        1.0       1.0   
1     699   37     175         1  1.142857  4.000000        1.0       1.0   
2     528   19      70         1  1.142857  3.918033        1.0       1.0   
3    1145   61      45         1  1.142857  3.918033        1.0       1.0   
4     517   41      28         1  1.142857  3.918033        1.0       1.0   

   Count_CL    ActBal_CA  ...    VolumeDeb_CA  VolumeDebCash_Card  \
0  1.000000     4.710714  ...      448.892857          178.571429   
1  1.285714  1823.057143  ...      566.126071           89.285714   
2  1.285714     0.000000  ...      390.056429          125.000000   
3  1.285714   324.710000  ...        3.392857            0.000000   
4  1.285714    17.051071  ...      557.896429           71.428571   

   VolumeDebCashless_Card  VolumeDeb_PaymentOrder  TransactionsDeb  \
0                0.000000              166.571429              8.0   
1                0.000000              216.892857             13.0   
2               70.842143              190.821429             10.0   
3                0.000000                0.000000              1.0   
4              224.146429              251.178571             17.0   

   TransactionsDeb_CA  TransactionsDebCash_Card  TransactionsDebCashless_Card  \
0                 7.0                       1.0                           0.0   
1                10.0                       2.0                           0.0   
2                10.0                       4.0                           3.0   
3                 1.0                       0.0                           0.0   
4                17.0                       2.0                          10.0   

   TransactionsDeb_PaymentOrder  Sex_F  
0                           4.0      0  
1                           5.0      0  
2                           2.0      1  
3                           0.0      0  
4                           3.0      1  

[5 rows x 30 columns]

In [63]:
# Saving the datasets
train_mf_final.to_csv(home_path+'train_mf_final.csv',index=False)
train_cc_final.to_csv(home_path+'train_cc_final.csv',index=False)
train_cl_final.to_csv(home_path+'train_cl_final.csv',index=False)
test_final.to_csv(home_path+'test_final.csv',index=False)

In [72]:
print(train_mf_final.shape)
print(train_cc_final.shape)
print(train_cl_final.shape)

(969, 32)
(969, 32)
(969, 32)


In [64]:
train_data.isnull().values.any()

False

In [65]:
test_data.isnull().values.any()

False

In [66]:
mf_target.isnull().values.any()

False

In [67]:
cc_target.isnull().values.any()

False

In [68]:
cl_target.isnull().values.any()

False

In [69]:
train_id.isnull().values.any()

False

In [70]:
test_id.isnull().values.any()

False

In [71]:
del train_id, train_data, test_data, test_id, mf_target, cc_target, cl_target